--- Day 6:  ---


In [361]:
class Folder:
    def __init__(self, name, parent):
        """
        A Folder with all its contained Files and Sub-Folders.

        Args:
            name str: Folder name
            parent Folder: Parent Folder class
        """
        self.name = name
        self.parent = parent
        self.child_folders = {}
        self.child_files = []
        if parent:
            self.path = parent.path + "/" + name
        else:
            self.path = ""

        self._size = None

    @property
    def child_dirs_list(self):
        return list(self.child_folders.values())

    @property
    def size(self):
        """ Caculate the size of this folder

        Returns:
            int: Folder size
        """
        if self._size:
            return self._size

        kids = self.child_dirs_list + self.child_files
        self._size = sum([child.size for child in kids])
        return self._size

    def get_all_child_dirs(self):
        """_summary_

        Returns:
            _type_: _description_
        """
        child_dirs_lists = [
            directory.get_all_child_dirs() for directory in self.child_dirs_list
        ]
        return self.child_dirs_list + sum(child_dirs_lists, [])

    def build_tree(self):
        """
        Build a dict tree of the files and folders
        """
        self.tree = {}
        for child in self.child_dirs_list:
            child.build_tree()
            self.tree[child.name] = child.tree
        for child in self.child_files:
            self.tree[child.name] = child.size

    def __str__(self):
        return self.path


In [362]:
class File:
    """ """

    def __init__(self, name, parent, size):
        """_summary_

        Args:
            name (str): File name
            parent (Folder): Parent Folder class
            size (int): Size of file
        """
        self.name = name
        self.parent = parent
        self.size = size

    def __str__(self):
        return self.name + " - " + str(self.size)


In [363]:
with open("d6_data.txt", "r") as file:
    data = file.read()

split_data = data.split("\n")
len(split_data)

1063

In [364]:
root_directory = Folder("/", None)
current_directory = None
rows = [row for row in split_data if row != "$ ls"]
for row in rows:
    if row.startswith("$ cd "):
        dir_name = row[5:]
        if dir_name == "/":
            current_directory = root_directory
        elif dir_name == "..":
            current_directory = current_directory.parent
        else:
            current_directory = current_directory.child_folders[dir_name]
    elif row.startswith("dir "):
        dir_name = row.split(" ")[1]
        current_directory.child_folders[dir_name] = Folder(dir_name, current_directory)
    else:
        file_details = row.split(" ")
        file_size = int(file_details[0])
        file_name = file_details[1]
        current_directory.child_files.append(
            File(file_name, current_directory, file_size)
        )


In [365]:
all_dirs = [root_directory] + root_directory.get_all_child_dirs()
folders_under_100000 = []
for folder in all_dirs:
    if folder.size <= 100000:
        folders_under_100000.append(folder)

print(sum([folder.size for folder in folders_under_100000]))


1581595


--- Part Two ---


In [366]:
space_to_free = root_directory.size - (70000000 - 30000000)
large_enough_dirs = [folder for folder in all_dirs if folder.size >= space_to_free]
large_enough_dirs.sort(key=lambda x: x.size)
best_dir = large_enough_dirs[0]
print(best_dir.size)


1544176
